In [1]:
import re
import time

import dateparser as dtparser
#from dateutil import parser as dtparser
from datetime import date

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen


# Грузим с банки.ру....

In [1]:
from banks import ClosedBanks

In [2]:
cl = ClosedBanks()

Processed 2 of 50...Index(['index', 'bank', 'license_number', 'reason_of_closing',
       'date_of_closing', 'city', 'link'],
      dtype='object')
РУБанк
Ханты-Мансийский банк Открытие
БайкалБанк
ГРиС-Банк
Русский Трастовый Банк
Приполярком
Газстройбанк
Промэнергобанк
Констанс-Банк
Экспо Финанс
Терра
Кредо Финанс
БФГ-Кредит
Кредит-Москва
АББ
Стратегия
Арксбанк
Ринвестбанк
ЕвроситиБанк
ПЧРБ Банк
Первобанк
Приско Капитал Банк
Интеркредит
Рускобанк
Бинбанк Смоленск
Межрегиональный Банк Реконструкции
Бинбанк Сургут
Кедр
КредитАльянс
Финансовый Стандарт
Промрегионбанк
Банк Развития Технологий
ЕвроАксис Банк
Верхневолжский
Век
Динамичные Системы
Эл Банк
Мострансбанк
Банк МБФИ
Интерактивный Банк
Инвестиционный Республиканский Банк
Стелла-Банк
Океан Банк
Пульс Столицы
Кроссинвестбанк
ФИА-Банк
Соверен Банк
Майма
Интеррасчет
Смартбанк


In [3]:
cl

,full_name,city,license_number,reason_of_closing,date_of_closing,reason_extended,index
0,АО «Коммерческий банк «РУБанк»,Москва,2813,отзыв лицензии,22.08.2016,Решение о применении крайней меры воздействия ...,600000004
0,ПАО «Ханты-Мансийский банк Открытие»,Москва,1971,решение о ликвидации,22.08.2016,Прекращение деятельности в связи с реорганизац...,710000048
0,ПАО «БайкалБанк»,Улан-Удэ,2632,отзыв лицензии,18.08.2016,Решение о применении крайней меры воздействия ...,810000006
0,ООО «Коммерческий банк «ГРиС-Банк»,Пятигорск,1928,отзыв лицензии,18.08.2016,Решение о применении крайней меры воздействия ...,70000005
0,АО «Акционерный Коммерческий Банк «Русский Тра...,Москва,1139,отзыв лицензии,16.08.2016,Решение о применении крайней меры воздействия ...,70000009
0,АО «Расчетная небанковская кредитная организац...,Тюмень,507-К,решение о ликвидации,12.08.2016,Приказом Банка России от 12.08.2016 N ОД-2616 ...,710000022
0,АО «Акционерный Коммерческий Банк «Газстройбанк»,Москва,2919,отзыв лицензии,12.08.2016,Решение о применении крайней меры воздействия ...,450001027
0,АО «Промышленный энергетический банк»,Вологда,2728,отзыв лицензии,05.08.2016,Решение о применении крайней меры воздействия ...,190000009
0,АО «Акционерный Коммерческий Банк «Констанс-Банк»,Санкт-Петербург,2228,отзыв лицензии,05.08.2016,Решение о применении крайней меры воздействия ...,410000008
0,АО «Банк «Экспо Финанс»,Москва,2594,решение о ликвидации,01.08.2016,Прекращение деятельности в связи с реорганизац...,450000518


In [26]:
#bank_definitions['index'] = bank_definitions['cbr_link'].map(lambda x: x[x.find('=')+1:])
bank_definitions.head()

,full_name,city,license_number,reason_of_closing,date_of_closing,reason_extended,index
0,АО «Коммерческий банк «РУБанк»,Москва,2813,отзыв лицензии,22.08.2016,Решение о применении крайней меры воздействия ...,600000004
0,ПАО «Ханты-Мансийский банк Открытие»,Москва,1971,решение о ликвидации,22.08.2016,Прекращение деятельности в связи с реорганизац...,710000048
0,ПАО «БайкалБанк»,Улан-Удэ,2632,отзыв лицензии,18.08.2016,Решение о применении крайней меры воздействия ...,810000006
0,ООО «Коммерческий банк «ГРиС-Банк»,Пятигорск,1928,отзыв лицензии,18.08.2016,Решение о применении крайней меры воздействия ...,70000005
0,АО «Акционерный Коммерческий Банк «Русский Тра...,Москва,1139,отзыв лицензии,16.08.2016,Решение о применении крайней меры воздействия ...,70000009


# Грузим с сайта ЦБ

In [4]:
from banks import Bank
from banks.utils import to_number
#bank = Bank(450000036,1326,'АЛЬФА-БАНК')
bank = Bank(450000370,2306,'Абсолют Банк')

In [53]:
def get_form135(self, first_n=None):
    soup = self._open_bank_page()

    reports =  soup.find('div' , {'class':'reports'})
    f_135 = reports.find('div', {'id':'f_135'})

    if f_135 is None:
        return pd.DataFrame()

    # Get list of reporting dates
    dates = []
    for el in f_135.find('div',{'class':'switched'}).findAll('div',{'class':'normal'}):
        year = el['id'][-4:]
        dates.extend([dtparser.parse(a.text[3:] + ' ' + year) for a in el.findAll('a')])


    f_135 = f_135.findAll('a')

    urls = [('http://www.cbr.ru/credit/'+a['href']).replace('®','&reg') for a in f_135]

    if first_n is None:
        first_n = len(urls)

    f_135 = pd.DataFrame()
    
    for url, date in zip(urls[:first_n], dates[:first_n]):

        page = urlopen(url).read()
        tables = pd.read_html(page)

        transcripts = tables[1].drop(0)
        norms = tables[2][[0,1]].drop(0).fillna(0)
        norms[1] = norms[1].map(lambda x: int(str(x))/100)
        print(norms)
        ul = BeautifulSoup(page, 'html.parser').find('ul', {'class':'without_dash without_indent'})
        indicators = []
        for li in ul.findAll('li'):
            li = ''.join(str(li.text).split())
            indicators.append([li[:li.find('=')],li[li.find('=')+1:]])

        indicators = pd.DataFrame(indicators)
        table = pd.concat([transcripts, norms, indicators]).set_index(0).fillna(0)
        table[1] = table[1].astype(np.str).map(to_number)
        table = table.T
        table['date'] = date

        f_135 = pd.concat([f_135, table]).fillna(0)
        
    f_135['index'] = self.bank_id
        
    return f_135

Bank.get_form135 = get_form135

f = bank.get_form135(1)


        0       1
1    Н1.1   10.34
2    Н1.2   10.34
3    Н1.0   12.79
4    Н1.3    0.00
5      Н2   58.53
6      Н3   84.26
7      Н4   83.62
8      Н7  323.93
9    Н9.1    0.00
10  Н10.1    1.11
11    Н12    1.05
12    Н15    0.00
13  Н15.1    0.00
14    Н16    0.00
15  Н16.1    0.00
16  Н16.2    0.00
17    Н18    0.00


In [36]:
f[['Н1.0', 'Н1.1', 'Н1.2', 'Н1.3', 'Н10.1', 'Н12', 'Н15',
       'Н15.1', 'Н16', 'Н16.1', 'Н16.2', 'Н18', 'Н2', 'Н3', 'Н4', 'Н7', 'Н9.1',]]

,Н1.0,Н1.1,Н1.2,Н1.3,Н10.1,Н12,Н15,Н15.1,Н16,Н16.1,Н16.2,Н18,Н2,Н3,Н4,Н7,Н9.1
1,1279.0,1034.0,1034.0,0.0,111.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,5853.0,8426.0,8362.0,32393.0,0.0


In [5]:
cbr_bank_list_url = 'http://www.cbr.ru/credit/transparent.asp'

page = urlopen(cbr_bank_list_url).read()

cbr_bank_list_df = pd.read_html(page)[0]
cbr_bank_list_df.drop([4,5,6],axis=1, inplace=True)
cbr_bank_list_df.columns = cbr_bank_list_df.ix[0]
cbr_bank_list_df.drop([0,1],axis=0, inplace=True)
cbr_bank_list_df.drop(['Раскрытие информации'],axis=1, inplace=True)

text = str(BeautifulSoup(page,'html.parser').find('table'))
ids = re.findall('javascript:info\((.+?)\)', text)

cbr_bank_list_df['№'] = ids
cbr_bank_list_df.columns = ['id', 'license_number', 'name']

cbr_bank_list = [Bank(bank.id,bank.license_number,bank.name) 
                 for bank in cbr_bank_list_df.itertuples(index=False)]

In [6]:

banks_main_info = pd.DataFrame()

counter = 1
total_spent = 0

form_101 = pd.DataFrame()
form_102 = pd.DataFrame()
form_123 = pd.DataFrame()
form_134 = pd.DataFrame()
form_135 = pd.DataFrame()


for bank in cbr_bank_list:
    
    start_time = time.time()
    
    print("{0} of {1} \t{2} - {3}".format(counter,
                                          len(cbr_bank_list),
                                          bank.bank_id, bank.name), end='')
    
    # downloading Form 101
    
    #f_101 = bank.get_form101(1)
    #form_101 = pd.concat([form_101,f_101])
    
    # downloading Form 102
    
    #f_102 = bank.get_form102(1)
    #form_102 = pd.concat([form_102,f_102])
    
    # downloading Form 123
    
    #f_123 = bank.get_form123(1)
    #form_123 = pd.concat([form_123,f_123])
    
    # downloading Form 134
    
    f_134 = bank.get_form123(1,'f_134')
    form_134 = pd.concat([form_134,f_134])
    # downloading Form 135
    
    #f_135 = bank.get_form135(1)
    #form_135 = pd.concat([form_135,f_135])
    
    total_spent += time.time()-start_time
    print('. Time spent - %d sec.' % (time.time()-start_time))
    counter += 1
    

1 of 643 	400000027 - 21 ВЕК. Time spent - 0 sec.
2 of 643 	450000370 - АБСОЛЮТ БАНК. Time spent - 1 sec.
3 of 643 	450000785 - АВАНГАРД. Time spent - 1 sec.
4 of 643 	920000004 - АВЕРС. Time spent - 0 sec.
5 of 643 	920000023 - АВТОГРАДБАНК. Time spent - 2 sec.
6 of 643 	920000009 - АВТОКРЕДИТБАНК. Time spent - 0 sec.
7 of 643 	450000618 - АВТОТОРГБАНК. Time spent - 1 sec.
8 of 643 	460000012 - АГРОПРОМКРЕДИТ. Time spent - 1 sec.
9 of 643 	630000015 - АГРОРОС. Time spent - 1 sec.
10 of 643 	530000007 - АГРОСОЮЗ. Time spent - 0 sec.
11 of 643 	100000003 - АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК. Time spent - 1 sec.
12 of 643 	450000753 - АЗИЯ БАНК. Time spent - 0 sec.
13 of 643 	450016176 - АЗИЯ-ИНВЕСТ БАНК. Time spent - 0 sec.
14 of 643 	450016204 - АЙВИ БАНК. Time spent - 0 sec.
15 of 643 	840000001 - АЙМАНИБАНК. Time spent - 0 sec.
16 of 643 	450054198 - АЙСИБИСИ БАНК. Time spent - 1 sec.
17 of 643 	920000005 - АК БАРС. Time spent - 1 sec.
18 of 643 	920000006 - АКИБАНК. Time spent - 0 sec.
19 

,Н1.0,Н1.1,Н1.2,Н1.3,Н10.1,Н12,Н15,Н15.1,Н16,Н16.1,Н16.2,Н18,Н2,Н3,Н4,Н7,Н9.1
1,6323.0,0.0,0.0,0.0,0.0,0.0,16361.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1279.0,1034.0,1034.0,0.0,111.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,5853.0,8426.0,8362.0,32393.0,0.0
1,1534.0,1377.0,1377.0,0.0,223.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,8851.0,8660.0,10799.0,19934.0,0.0
1,3929.0,3562.0,3562.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7625.0,13044.0,4203.0,9585.0,1.0
1,1648.0,1127.0,1127.0,0.0,166.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9082.0,10449.0,8042.0,14190.0,8.0
1,10371.0,9434.0,9434.0,0.0,187.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2963.0,23471.0,2645.0,8767.0,1875.0
1,3250.0,1074.0,1074.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9569.0,11559.0,6434.0,10357.0,0.0
1,1462.0,1462.0,1462.0,0.0,142.0,1670.0,0.0,0.0,0.0,0.0,0.0,0.0,26094.0,18280.0,3507.0,24473.0,0.0
1,1775.0,1338.0,1338.0,0.0,143.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15382.0,10887.0,4872.0,12541.0,227.0
1,1189.0,1041.0,1041.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20424.0,25775.0,3404.0,43283.0,0.0
